<a href="https://colab.research.google.com/github/audwls4545/metaverse/blob/main/pytorch_%EC%A3%BC%EC%8B%9Dlstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_datareader==0.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: pandas_datareader
    Found existing installation: pandas-datareader 0.10.0
    Uninstalling pandas-datareader-0.10.0:
      Successfully uninstalled pandas-datareader-0.10.0


In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt

from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader,TensorDataset
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import yfinance as yf

from datetime import date,timedelta

today = date.today()
end_date = today.strftime('%Y-%m-%d')

d1 = date.today() - timedelta(days=360*5)
start_date = d1.strftime('%Y-%m-%d')

df = yf.download(tickers = '005930.KS',
                 start = start_date,
                 end = end_date
                 )

df = df.iloc[:,0:4]

[*********************100%%**********************]  1 of 1 completed


In [ ]:
seq_length = 7
batch = 50
train_size = int(len(df)*0.7)
train_set = df[0:train_size]
test_set = df[train_size-seq_length : ]

In [ ]:
train_set

,Open,High,Low,Close
Date,,,,
2018-10-10,45250.0,45500.0,44500.0,45300.0
2018-10-11,44000.0,44650.0,43100.0,43100.0
2018-10-12,43200.0,44650.0,43200.0,44000.0
2018-10-15,44050.0,44050.0,43350.0,43800.0
2018-10-16,43700.0,44150.0,43350.0,43600.0
...,...,...,...,...
2022-03-18,70600.0,70900.0,70200.0,70700.0
2022-03-21,70900.0,71000.0,69900.0,69900.0
2022-03-22,69900.0,70500.0,69900.0,70300.0


In [ ]:
test_set

,Open,High,Low,Close
Date,,,,
2022-03-16,70200.0,70500.0,69700.0,70400.0
2022-03-17,70400.0,70400.0,70400.0,70400.0
2022-03-18,70600.0,70900.0,70200.0,70700.0
2022-03-21,70900.0,71000.0,69900.0,69900.0
2022-03-22,69900.0,70500.0,69900.0,70300.0
...,...,...,...,...
2023-09-07,70000.0,70600.0,69600.0,70400.0
2023-09-08,70200.0,70300.0,69600.0,70300.0
2023-09-11,70400.0,70800.0,70000.0,70800.0


In [ ]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_set.iloc[:,:-1])

train_set.iloc[:,:-1] = scaler_x.transform(train_set.iloc[:,:-1])
test_set.iloc[:,:-1]  = scaler_x.transform(test_set.iloc[:,:-1])

<ipython-input-12-317b0d5d1e58>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.iloc[:,:-1] = scaler_x.transform(train_set.iloc[:,:-1])
<ipython-input-12-317b0d5d1e58>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set.iloc[:,:-1]  = scaler_x.transform(test_set.iloc[:,:-1])


In [ ]:
train_set

,Open,High,Low,Close
Date,,,,
2018-10-10,0.147588,0.133446,0.145299,45300.0
2018-10-11,0.123936,0.119088,0.118708,43100.0
2018-10-12,0.108798,0.119088,0.120608,44000.0
2018-10-15,0.124882,0.108953,0.123457,43800.0
2018-10-16,0.118259,0.110642,0.123457,43600.0
...,...,...,...,...
2022-03-18,-0.708597,-0.635126,-0.699893,70700.0
2022-03-21,-0.708597,-0.635126,-0.699893,69900.0
2022-03-22,-0.708598,-0.635126,-0.699893,70300.0


In [ ]:
scaler_y = MinMaxScaler()
scaler_y.fit(train_set.iloc[:,[-1]])

train_set.iloc[:,-1] = scaler_y.transform(train_set.iloc[:,[-1]])
test_set.iloc[:,-1] = scaler_y.transform(test_set.iloc[:,[-1]])

<ipython-input-15-d791cf66bcde>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.iloc[:,-1] = scaler_y.transform(train_set.iloc[:,[-1]])
<ipython-input-15-d791cf66bcde>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set.iloc[:,-1] = scaler_y.transform(test_set.iloc[:,[-1]])


In [ ]:
np.array(train_set)

array([[ 0.14758751,  0.13344595,  0.14529915,  0.14659197],
       [ 0.12393567,  0.11908784,  0.11870845,  0.10550887],
       [ 0.10879849,  0.11908784,  0.12060779,  0.12231559],
       ...,
       [-0.70859765, -0.63512575, -0.69989311, -0.69933495],
       [-0.7085974 , -0.63512555, -0.69989297, -0.69933488],
       [-0.70859776, -0.6351258 , -0.69989322, -0.69933512]])

In [ ]:
#데이터셋 생성 함수

def build_dataset(time_series,seq_length):
    dataX = []
    dataY = []

    for i in range(0, len(time_series) - seq_length):
        x = time_series[i:i+seq_length,:]
        y = time_series[i+seq_length , [-1]]

        dataX.append(x)
        dataY.append(y)

    return np.array(dataX) , np.array(dataY)


trainX,trainY = build_dataset(np.array(train_set) , seq_length)
testX,testY =  build_dataset(np.array(test_set) , seq_length)

In [ ]:
trainX.shape

(845, 7, 4)

In [ ]:
trainY.shape

(845, 1)

In [ ]:
trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)
testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)

In [ ]:
dataset = TensorDataset(traninX_tensor, trainY_tensor)

dataloader = DataLoader(dataset, batch_size=batch, shuffle=True, drop_last=True)